In [208]:
import json
import numpy as np
import pandas as pd 
import seaborn as sns
from commands import COMMANDS
from matplotlib import pyplot as plt

sns.set_style('whitegrid')

In [209]:
data = pd.read_csv('../data/ad_music_proccessed.csv', index_col=0)
leads = pd.read_csv('../data/leads.csv' , index_col=0)

In [210]:
phising_lead = leads.iloc[1]
phising_lead[phising_lead != 0]

cookie                               6de19499161f9da71032f40b8dd63eb1
unique_user_ids     ['U772698', 'U038703', 'U736144', 'U650933', '...
num_users                                                          15
target_artist                                                   [nan]
target_user         [nan, 'U814351', 'U215441', 'U067778', 'U63557...
num_target_users                                                  340
change_password                                                    15
disable_mfa                                                        15
homepage                                                           28
personal_details                                                   15
send_message                                                      339
Name: 1, dtype: object

In [211]:
data[data.cookie.eq(phising_lead.cookie)].command.value_counts()

command
send_message        339
homepage             28
personal_details     15
change_password      15
disable_mfa          15
Name: count, dtype: int64

In [212]:
ck_data = data[data.cookie.eq(phising_lead.cookie)]
ck_data = ck_data[ck_data.command.isin(COMMANDS.ato)]
user_command_count = ck_data.groupby('user_id')['command'].nunique()
qualified_user_ids = user_command_count[user_command_count.eq(2)].index
len(qualified_user_ids)

15

The cookie '6de19499161f9da71032f40b8dd63eb1' has 15 accounts linked to it.
All of them changed passwords and disabled MFA - so they are hijacked (ATO).

In [213]:
gp = data[data.user_id.isin(qualified_user_ids)]
gp =  gp[gp.command.eq('send_message')]
gp.groupby(gp.user_id)['command'].count()

user_id
U038703    15
U155816    30
U175180    24
U181694    24
U217250    26
U233527    22
U294634    13
U445622    24
U493261    23
U650933    28
U736144    21
U741536    21
U752499    19
U772698    23
U956095    26
Name: command, dtype: int64

these users have sent 339 messages to 339 users - therefore I assume they were all used to phish.
the affected accounts are:

In [214]:
print(phising_lead.target_user)

[nan, 'U814351', 'U215441', 'U067778', 'U635572', 'U261904', 'U072722', 'U882040', 'U221352', 'U653192', 'U275033', 'U235310', 'U901110', 'U150188', 'U126768', 'U109070', 'U461332', 'U887018', 'U803632', 'U398953', 'U228861', 'U363593', 'U339709', 'U446002', 'U121466', 'U620898', 'U998497', 'U958167', 'U592260', 'U366410', 'U683448', 'U370021', 'U721406', 'U772497', 'U276601', 'U909192', 'U104275', 'U652643', 'U687168', 'U695316', 'U123850', 'U596419', 'U539135', 'U051240', 'U796018', 'U521003', 'U099438', 'U707641', 'U365585', 'U962271', 'U867970', 'U417092', 'U635376', 'U592182', 'U938141', 'U308624', 'U684029', 'U532230', 'U238855', 'U575832', 'U183676', 'U481254', 'U515754', 'U739833', 'U957025', 'U303617', 'U511242', 'U255119', 'U708566', 'U567187', 'U388556', 'U125944', 'U608409', 'U970013', 'U807499', 'U798657', 'U767633', 'U995710', 'U234599', 'U923534', 'U115195', 'U823623', 'U032570', 'U716748', 'U569628', 'U352329', 'U799714', 'U391599', 'U155678', 'U991957', 'U335264', 'U89

In [215]:
# Save affected Accounts
out = phising_lead.target_user.split(', ')[1:]
out = [x.replace("'","") for x in out]
out = [x.replace("]","") for x in out]
out_ato = qualified_user_ids.values.tolist()

with open("affected_users.json", "r") as f:
    out_file = json.load(f)

out_file['phishing'] = out
if 'ato' in out_file.keys():
    out_file['ato'] = out_file['ato'] + out_ato
else:
    out_file['ato'] = out_ato
with open("affected_users.json", "w") as f:
    json.dump(out_file, f, indent=4)

we can search for evidence of phishing in both general ato population and free account population.

free accounts

In [216]:
free_accounts = data[data.subscription_status.eq('trial')]
free_accounts.shape

(6662, 34)

In [217]:
msg_accounts = free_accounts[free_accounts.command.eq('send_message')]
msg_accounts.user_id.nunique()

45

45 free users have sent messages.

In [218]:
msg_accounts.email_domain.value_counts()

email_domain
yahoo.com      44
hotmail.com    25
gmail.com      20
Name: count, dtype: int64

In [219]:
msg_accounts.ip_c_class.value_counts()

ip_c_class
88.0.0     28
79.0.0     17
68.0.0     17
12.0.0     10
217.0.0    10
73.0.0      7
Name: count, dtype: int64

In [220]:
msg_accounts.os_family.value_counts()

os_family
Mac OS X    32
Linux       31
Windows     26
Name: count, dtype: int64

In [221]:
msg_accounts[msg_accounts.browser_family.eq('Edge')][msg_accounts.os_family.eq('Linux')]

/var/folders/ms/zw030byx3qsfvlvyyfnnr7sm0000gn/T/ipykernel_33556/1831712990.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  msg_accounts[msg_accounts.browser_family.eq('Edge')][msg_accounts.os_family.eq('Linux')]


,timestamp,user_id,user_email,account_creation_date,subscription_status,account_tier,path,http_method,ip,org,...,domain_rarity,ip_c_class,device_age,cookie_age,country_age,asn_age,ua_age,browser_age,os_age,ip_age
86922,2024-07-01 22:42:58,U572283,michaelwilson@gmail.com,2024-07-01 22:09:43,trial,listener,/send_to/U847456/send_message,POST,73.0.0.53,"Comcast Cable Communications, LLC",...,0.324374,73.0.0,0,0,0,0,0,0,0,0
87068,2024-07-02 00:40:08,U387239,ianchavez@yahoo.com,2024-07-01 23:59:43,trial,listener,/send_to/U276679/send_message,POST,88.0.0.90,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0
88610,2024-07-03 09:22:44,U390856,qjohnson@yahoo.com,2024-07-01 16:58:18,trial,listener,/send_to/U600173/send_message,POST,88.0.0.56,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0
88781,2024-07-03 12:55:06,U943901,herrerajessica@yahoo.com,2024-07-01 20:02:26,trial,listener,/send_to/U074905/send_message,POST,88.0.0.30,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0
88833,2024-07-03 13:44:42,U390856,qjohnson@yahoo.com,2024-07-01 16:58:18,trial,listener,/send_to/U696726/send_message,POST,88.0.0.56,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0
90856,2024-07-05 08:03:12,U390856,qjohnson@yahoo.com,2024-07-01 16:58:18,trial,listener,/send_to/U952291/send_message,POST,88.0.0.56,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0
90992,2024-07-05 11:36:32,U157986,roy08@hotmail.com,2024-07-04 00:00:00,trial,listener,/send_to/U763012/send_message,POST,217.0.0.63,British Telecommunications PLC,...,0.310417,217.0.0,15,30,10,26,34,34,34,23
91359,2024-07-05 18:24:29,U362494,williamhanson@yahoo.com,2024-07-01 02:19:08,trial,listener,/send_to/U520387/send_message,POST,79.0.0.3,Telecom Italia,...,0.317860,79.0.0,0,0,0,0,0,0,0,0
91669,2024-07-06 00:47:06,U362494,williamhanson@yahoo.com,2024-07-01 02:19:08,trial,listener,/send_to/U563710/send_message,POST,79.0.0.3,Telecom Italia,...,0.317860,79.0.0,0,0,0,0,0,0,0,0
91788,2024-07-06 03:25:04,U387239,ianchavez@yahoo.com,2024-07-01 23:59:43,trial,listener,/send_to/U348649/send_message,POST,88.0.0.90,Telefonica de Espana,...,0.317860,88.0.0,0,0,0,0,0,0,0,0


After futher examination of the data i saw no evidence of malicious activity.
These accounts use legit email domains (hotmail, gmail, yahoo), spread over different ip a classes, different devices and browsers. 
Even the strage case of Edge browser on Linux seems legit. 